# Make ASCE_SIFT_Region2 dtopo file

This Jupyter notebook is available in `$GTT/dtopo/ASCE_SIFT_Region2.ipynb` within the
[GeoClaw Tsunami Tutorial](https://rjleveque.github.io/geoclaw_tsunami_tutorial).

It creates the file `$GTT/dtopo/dtopofiles/ASCE_SIFT_Region2.dtt3`, which can be used as a [dtopofile](https://www.clawpack.org/dtopo.html) in GeoClaw. 

:::{admonition}To Do
:class: dropdown
- Add more description of ASCE, SIFT, unit sources
- Plot transects
- Make dtopo contours in kmz file
:::

:::{seealso}
- [dtopotools_examples notebook](https://www.clawpack.org/gallery/_static/apps/notebooks/geoclaw/dtopotools_examples.html)
- [Earthquake sources: Fault slip and the Okada model](https://www.clawpack.org/okada.html)
- [Okada notebook](https://www.clawpack.org/gallery/_static/apps/notebooks/geoclaw/Okada.html)
:::

Rather than building this dtopofile, it is also possible to download it using `GTT/dtopo/fetch_ASCE_SIFT_Region2.py`.


### Setup notebook and environment:

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import dtopotools, topotools, kmltools

#### Load shoreline for plotting:

This file was created with the [](../topo/fetch_etopo22) notebook.

In [ ]:
csz_shore = loadtxt('../topo/topofiles/csz_shore.txt')

## Create a fault using the NOAA SIFT database

This particular fault is specified in terms of the NOAA SIFT unit sources, so we can use the [clawpack.geoclaw.dtopotools.SiftFault](https://www.clawpack.org/dtopotools_module.html#clawpack.geoclaw.dtopotools.SiftFault) class, which takes as an argument a dictionary *sift_slip* specifying the unit sources to be used and the slip on each.  

First we plot some of the unit sources on the CSZ margin.

In [ ]:
f = dtopotools.SiftFault()
SIFT = f.sift_subfaults
sift_slips = {}
for k in SIFT.keys():
    s = SIFT[k]  # subfault
    s.longitude = s.longitude - 360.
    if (-130 <=s.longitude <= -120) and (44 <= s.latitude <= 50):
        if (k[4] in ['a','b']):
            sift_slips[k] = 0.

In [ ]:
csz = dtopotools.SiftFault(sift_slips)
for k in sift_slips.keys():
    s = csz.sift_subfaults[k]  # subfault
    s.longitude = s.longitude - 360.

figure(figsize=(6,6))
ax = axes()
ax.plot(csz_shore[:,0], csz_shore[:,1], 'g', linewidth=0.7)
#axis([-140,-110,43,50])
ax.set_xlim(-130,-120)
ax.set_ylim(44,50)
csz.plot_subfaults(axes=ax)
ax.set_aspect(1./cos(55*pi/180.))

for k in sift_slips.keys():
    s = csz.sift_subfaults[k]  # subfault
    if k[4] == 'a':
        text(s.longitude, s.latitude, k, color='r')
    else:
        text(s.longitude-2, s.latitude, k, color='r')

### Set the slip on a set of SIFT subfaults for this event

In [ ]:
event = 'ASCE_SIFT_Region2'

Slips provided by Yong Wei: 35*ac57a+35*ac57b+10*ac58a+33*ac58b+10*ac59a+24*ac59b+30*ac60a+30*ac60b

In [ ]:
sift_slips = {'acsza57':35, 'acszb57':35 , 'acsza58':10 , 'acszb58':33 , 'acsza59':10 , 'acszb59':24 ,
              'acsza60':30 , 'acszb60':30 }
fault = dtopotools.SiftFault(sift_slips)
for k in sift_slips.keys():
    s = fault.sift_subfaults[k]  # subfault
    s.longitude = s.longitude - 360.

### Create a uniform grid on which to evaluate the vertical deformation

The grid spacing will be 10 arcseconds, with a buffer of 1 degree around the horizontal extent of the subfaults.  (You could specify `x,y` to be any other desired points to define the deformation grid.) 

In [ ]:
x,y = fault.create_dtopo_xy(dx=10./3600., buffer_size=1)
print('Created uniform grid with %i points in x and %i in y' \
        % (len(x), len(y)))

### Apply the Okada model to each of the subfaults

Okada applied to each subfault, with the specified slip, gives a vertical deformation that is evaluated at all points on the grid defined by `x, y`.  These are added together for all the subfaults, since the Okada model is linear.

In [ ]:
print('Creating dtopo by applying Okada model to %s subfaults...' % len(fault.subfaults))
dtopo_SIFT = fault.create_dtopography(x,y, verbose=False)

## Plot the slip distribution and the resulting vertical deformation dz

In [ ]:
# for setting color scale:
dz_max = abs(dtopo_SIFT.dZ).max()
print("maximum abs(dz) over the full rupture time:", \
    abs(dtopo_SIFT.dZ).max())

fig = figure(figsize=(12,6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
fault.plot_subfaults(axes=ax1, slip_color=True)
dtopo_SIFT.plot_dZ_colors(axes=ax2, t=1, cmax_dZ=dz_max)

ax1.plot(csz_shore[:,0], csz_shore[:,1], 'g', linewidth=0.7)
ax1.set_xlim(-130,-120)
ax1.set_ylim(42,50)
ax1.set_aspect(1./cos(55*pi/180.))

ax2.plot(csz_shore[:,0], csz_shore[:,1], 'g', linewidth=0.7)
ax2.set_xlim(-130,-120)
ax2.set_ylim(42,50)
ax2.set_aspect(1./cos(55*pi/180.));

fname = 'dtopofiles/%s.png' % event
savefig(fname, bbox_inches='tight')
print('Created ',fname)

## Save as a dtopo file

In [ ]:
fname = 'dtopofiles/%s.dtt3' % event
dtopo_SIFT.write(fname, dtopo_type=3)
print('Created ',fname)

### Make a kmz file that shows contours of dtopo:

In [ ]:
fname_kmz = 'dtopofiles/%s_contours.kmz' % event

kmltools.dtopo_contours2kmz([fname], dtopo_type=3, 
                            dZ_interval=2, dZmax=20, 
                            text_label=True, 
                            text_x=dtopo_SIFT.x.min(),
                            text_y=dtopo_SIFT.y.mean(),
                            fname_kmz=fname_kmz)


Open the resulting file `dtopofiles/ASCE_SIFT_Region2_contours.kmz` in Google Earth to see contours of the vertical deformation.